In [2]:
import pandas as pd
import json

from tqdm import tqdm
from typing import List, Dict
from pymongo import MongoClient
import boto3

# Data Structure Observation

The primary objective of this notebook is to observe the structure of the data in the database.

Let's retrieve some data to better understand their structure.

- The data is stored in MongoDB
- Each feedback is a "document" that can contain multiple fields/metadata
- The mandatory metadata are:
    * Timestamp
    * A text field (often verbatim)
    * An ID
    * A brand (to filter our data based on the client)

We will take the example of the brand: ColumbusCafe

Dashboard link :  https://dashboard.allobrain.com/columbuscafe?filter_time_range=%5B1660521600000%2C1727733599999%5D&

Connect to the database

In [3]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING)

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

Brand:

In [31]:
BRAND = 'columbuscafe_test'

In [32]:
# getting 100 documents from picard brand

from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
    { "$sample" : { "size": 100 } }
])))

from_mongo.shape

(100, 21)

In [33]:
from_mongo.head()

,_id,id,brand,timestamp,verbatim,extractions,splitted_analysis_v2,topics_v2,Note Satisfaction,origin,...,Canal,Country,Objectifs,Type de visiteurs,Web / App 2024,CES,Centre de contact,Etoile,NPS,rating
0,sncf_connect_ao/aaae1add9addbfa0b0a7,aaae1add9addbfa0b0a7,sncf_connect_ao,1709247600000,{'text': 'Rapide et conviviale Pas facile pour...,"[{'extraction': 'Rapidité', 'sentiment': 'POSI...","[{'text': 'Rapide et conviviale ', 'extraction...",{},9.0,Enquêtes de satisfaction : SAT 2024,...,Enquêtes de satisfaction > SAT 2024,France,[Consulter l'info trafic ou les horaires],Utilisateur,App,NaN,None,NaN,NaN,NaN
1,sncf_connect_ao/97978f3bc72a88dc936c,97978f3bc72a88dc936c,sncf_connect_ao,1709247600000,{'text': 'Satisfait de l'application'},[{'extraction': 'Satisfaction avec l'applicati...,"[{'text': 'Satisfait de l'application', 'extra...",{},NaN,Mobile stores : SNCF Connect FR App store,...,Mobile stores > SNCF Connect FR App store,None,None,None,None,NaN,None,5.0,NaN,5.0
2,sncf_connect_ao/59669af5cb411db2c1ef,59669af5cb411db2c1ef,sncf_connect_ao,1704063600000,{'text': 'J'ai apprécié la rapidité de la répo...,"[{'extraction': 'Rapidité de la réponse', 'sen...",[{'text': 'J'ai apprécié la rapidité de la rép...,{},10.0,Enquêtes de satisfaction : SAT offline (RC),...,Enquêtes de satisfaction > SAT offline (RC),None,None,None,None,1.0,Comdata privé,NaN,10.0,NaN
3,sncf_connect_ao/f21dad7cd109b6905f48,f21dad7cd109b6905f48,sncf_connect_ao,1704063600000,"{'text': 'Depuis la dernière mise à jour, obli...",[{'extraction': 'Obligation de saisir à nouvea...,"[{'text': 'Depuis la dernière mise à jour,'}, ...",{'Parcours achat XSELL (pré-achat) > Dématéria...,NaN,Mobile stores : SNCF Connect FR App store,...,Mobile stores > SNCF Connect FR App store,None,None,None,None,NaN,None,2.0,NaN,2.0
4,sncf_connect_ao/818252c050da0b1503f1,818252c050da0b1503f1,sncf_connect_ao,1704063600000,{'text': ' Application très rapide et simple.'},[{'extraction': 'Application très rapide et si...,[{'text': ' Application très rapide et simple....,{'Parcours achat TRAIN (pré-achat) > Alerte pe...,NaN,Mobile stores : SNCF Connect FR Google Play,...,Mobile stores > SNCF Connect FR Google Play,None,None,None,None,NaN,None,5.0,NaN,5.0


In [34]:
sample_document = from_mongo.sample().iloc[0]

# the text of the client feedback
print(sample_document.verbatim)

{'text': "Application assez intuitive\nEn voulant prendre un billet je me suis aperçu que ma date de naissance a été modifiée au 16/11/2023. Date qui correspond à l'achat de la carte remi liberté. J'ai pas pu réserver car les réductions ne s'appliquaient pas. Pour modifier ma date de naissance il a fallu aller sur itou... bref le changement n'a pas été fait de suite. Internet c'est bien mais rien ne vaut le contact humain. Ce matin je suis allée à la gare de dreux pour effectuer ce changement le plus rapidement possible. Un grand merci à Frank qui a pu résoudre ce souci de date\n"}


### Exploring the different fields of the document


In [35]:
print("Timestamp:", sample_document.timestamp)
print("Text field:", sample_document.verbatim)
print("ID:", sample_document._id)
print("Brand:", sample_document.brand)

Timestamp: 1706742000000
Text field: {'text': "Application assez intuitive\nEn voulant prendre un billet je me suis aperçu que ma date de naissance a été modifiée au 16/11/2023. Date qui correspond à l'achat de la carte remi liberté. J'ai pas pu réserver car les réductions ne s'appliquaient pas. Pour modifier ma date de naissance il a fallu aller sur itou... bref le changement n'a pas été fait de suite. Internet c'est bien mais rien ne vaut le contact humain. Ce matin je suis allée à la gare de dreux pour effectuer ce changement le plus rapidement possible. Un grand merci à Frank qui a pu résoudre ce souci de date\n"}
ID: sncf_connect_ao/7e7b522ae43ef309cec5
Brand: sncf_connect_ao


### Checking for additional fields

Each brand has its own metadata !

In [ ]:
print(sample_document.rating_out_of_5)
print(sample_document.establishment)
print(sample_document.author)

## Analysis fields

### 1. Topic Extraction

Firstly, we will extract the topics that emerge from a review. The goal is to transform a long text containing several mixed topics into a list of distinct and reformulated topics.

Each extracted topic can have a positive, negative, or neutral sentiment associated with it. The sentiment of each topic is indicated in the "sentiment" field.

In [37]:
print(sample_document.verbatim['text'])

sample_document.extractions

Application assez intuitive
En voulant prendre un billet je me suis aperçu que ma date de naissance a été modifiée au 16/11/2023. Date qui correspond à l'achat de la carte remi liberté. J'ai pas pu réserver car les réductions ne s'appliquaient pas. Pour modifier ma date de naissance il a fallu aller sur itou... bref le changement n'a pas été fait de suite. Internet c'est bien mais rien ne vaut le contact humain. Ce matin je suis allée à la gare de dreux pour effectuer ce changement le plus rapidement possible. Un grand merci à Frank qui a pu résoudre ce souci de date



[{'extraction': 'Application assez intuitive',
  'sentiment': 'POSITIVE',
  'elementary_subjects': ["Interface intuitive et conviviale de l'application"],
  'topics': []},
 {'extraction': 'Modification de la date de naissance non immédiate',
  'sentiment': 'NEGATIVE',
  'elementary_subjects': ['Problèmes de synchronisation et de mise à jour des données'],
  'topics': [{'topic_lvl1': 'Parcours achat XSELL (pré-achat)',
    'topic_lvl2': 'Dématérialisation'}]},
 {'extraction': "Impossibilité d'appliquer les réductions suite à la modification de la date de naissance",
  'sentiment': 'NEGATIVE',
  'elementary_subjects': ["Incompatibilité des réductions sur l'application"],
  'topics': [{'topic_lvl1': 'Parcours achat TRAIN (pré-achat)',
    'topic_lvl2': 'Politique tarifaire'}]},
 {'extraction': "Préférence pour le contact humain plutôt que l'application",
  'sentiment': 'NEUTRAL',
  'suggestions_subjects': ['Amélioration de la communication humaine du service client']}]

In [28]:
for extr in sample_document.extractions:
    print('Extraction:', extr['extraction'])
    print('Sentiment:', extr['sentiment'])
    print('-' * 50)

Extraction: Service de mauvaise qualité
Sentiment: NEGATIVE
--------------------------------------------------
Extraction: Incompétence de la personne à la caisse
Sentiment: NEGATIVE
--------------------------------------------------
Extraction: Erreurs dans les commandes
Sentiment: NEGATIVE
--------------------------------------------------
Extraction: Manque de communication sur les choix de lait
Sentiment: NEGATIVE
--------------------------------------------------
Extraction: Service très long
Sentiment: NEGATIVE
--------------------------------------------------
Extraction: Problèmes avec la remise des aliments après paiement
Sentiment: NEGATIVE
--------------------------------------------------


**Detailed Structure of Extractions**

Upon closer examination, each object within the "splitted_analysis_v2" field contains the following information:

- Extraction: The extracted topic or subject.
- Sentiment: The sentiment associated with the extraction (positive, negative, or suggestion).
- Elementary Subjects: Generated subjects that allow us to classify the extractions.
- Topics (optional): More general and business-oriented subjects.

#### Elementary Subjects

Elementary subjects are generated subjects that help us classify the extractions. They are designed to highlight the most frequent subjects expressed by customers. These elementary subjects are displayed in the "Top Subjects" graph on the dashboard.

The purpose of elementary subjects is to provide a structured and organized way to categorize the extracted topics. By identifying common themes and grouping similar extractions together, we can gain insights into the most prevalent issues or opinions expressed by customers.

#### Topics

Topics, on the other hand, are more general and business-oriented subjects. They are less numerous compared to elementary subjects and provide a higher-level categorization.

Topics are intended to capture broader themes or categories that are relevant to the business or domain. They allow for a more strategic view of the feedback and can help identify overarching areas of concern or satisfaction.

In [27]:
for extr in sample_document.extractions:
    print('Extraction:', extr['extraction'])
    print('Elementary subjects:', extr['elementary_subjects'])
    print('Topics:', extr['topics'])
    print('-' * 50)

Extraction: Service de mauvaise qualité
Elementary subjects: ['Mauvaise qualité du service']
Topics: []
--------------------------------------------------
Extraction: Incompétence de la personne à la caisse
Elementary subjects: ['Incompétence ou manque de motivation du personnel']
Topics: [{'topic_lvl1': 'Le personnel', 'topic_lvl2': 'Amabilité du personnel'}]
--------------------------------------------------
Extraction: Erreurs dans les commandes
Elementary subjects: ['Erreurs fréquentes dans les commandes']
Topics: [{'topic_lvl1': 'Le personnel', 'topic_lvl2': 'Rapidité du service'}]
--------------------------------------------------
Extraction: Manque de communication sur les choix de lait
Elementary subjects: ['Manque d information sur les allergies et choix de produits']
Topics: []
--------------------------------------------------
Extraction: Service très long
Elementary subjects: ['Attente prolongée pour passer commande']
Topics: [{'topic_lvl1': 'Le personnel', 'topic_lvl2': 'R

### 2. Linking Extractions to Feedback

Each extracted topic is linked to the corresponding part of the feedback in the "splitted_analysis_v2" field. This field allows us to highlight the topics in the "Details" graph on the dashboard.

The "splitted_analysis_v2" field contains information that maps the extracted topics to their respective positions within the original feedback text. This mapping enables us to visually highlight the relevant parts of the feedback when displaying the extracted topics on the dashboard.

In [23]:
sample_document.splitted_analysis_v2

[{'text': 'Le service laisse largement à désirer.',
  'extraction': 'Service de mauvaise qualité',
  'sentiment': 'NEGATIVE',
  'topics': []},
 {'text': " La personne a la caisse n'est pas du tout compétente.",
  'extraction': 'Incompétence de la personne à la caisse',
  'sentiment': 'NEGATIVE',
  'topics': ['Le personnel > Amabilité du personnel']},
 {'text': ' Nous avons du redonner nos boisson trois fois pour enfin être servie.'},
 {'text': " J'ai demander un frappé et l'on ne m'a même pas demander le type de lait que je voulais.",
  'extraction': 'Manque de communication sur les choix de lait',
  'sentiment': 'NEGATIVE',
  'topics': []},
 {'text': ' Je ne digèrent pas le lait de vache.'},
 {'text': " J'ai donc dû demander moi même à une des serveuses après avoir entendue des clients en parler à une table.\n"},
 {'text': 'Le services à été vraiment très long.',
  'extraction': 'Service très long, Erreurs dans les commandes',
  'sentiment': 'NEGATIVE',
  'topics': ['Le personnel > Ra